In [ ]:
import os 
import re
import json
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as smt

def sin_cos_24hr_encoder(df, col):
    """
    Given a dataframe and column 
    This function takes a dataframe and a column that contains hours of the day,
    and creates two new columns 'Hour_Sin' and 'Hour_Cos'. These columns contain
    the sine and cosine encodings of the time, which is a common technique used
    for cyclical features like time of day in machine learning models.

    The sine and cosine values are calculated such that they form a circle when plotted,
    which is useful for the model to recognize the cyclical nature of the data
    (e.g., 23 hours is close to 0 hours).

    Parameters:
    df (DataFrame): The pandas dataframe to process.
    col (str): The column name with the hour values (0 to 23).

    Returns:
    DataFrame: The dataframe with two additional columns 'Hour_Sin' and 'Hour_Cos'.
    """
    time_delta_hours_sin = np.sin(2 * np.pi * df[col] / 24)
    time_delta_hours_cos = np.cos(2 * np.pi * df[col] / 24)
    df['Hour_Sin'] = time_delta_hours_sin
    df['Hour_Cos'] = time_delta_hours_cos

    return df

def sin_cos_24hr_decoder(df, sin_t, cos_t):
    """
    This function takes a dataframe and the column names for sine and cosine
    values that represent time. It calculates the corresponding hour of the day
    for each row and creates a new column 'hour_int' with these values.
    
    Parameters:
    df (DataFrame): The pandas dataframe to process.
    sin_t (str): The column name for the sine of time values.
    cos_t (str): The column name for the cosine of time values.
    
    Returns:
    DataFrame: The dataframe with an additional 'hour_int' column.
    """
    # Compute the angle in radians from the sin and cos values
    angle_rad = np.arctan2(df[sin_t], df[cos_t])
    
   # Ensure the angle is positive for all values
    angle_rad[angle_rad < 0] += 2 * np.pi
    
    # Convert the angle in radians to hours and add as a new column
    df['hour_int'] = round(angle_rad / (2 * np.pi) * 24, 3)
    
    return df

def construct_cellbox_input(Perturbations, Molecular, Phenotype, Output_path):
    """
    Load Perturbation, Molecular, and Phenotype data 
    Where the samples correspond to column names and rows correspond to nodes in the Cellbox output
    
    Parameters:
        Perturbation: Changing experimental conditions 
        Molecular: Measured cellular response 
        Phenotype: Measured output corresponding to phenotype of interest
        Output_Directory: Where the results should be written
        
    Returns:
        expression_matrix: a matrix of values corresponding to the indivual input values compiled for cellboxs
        perturbation_matrix: a matrix of values corresponding to initial base condition values (0) and associated perturbations
        node_index: the names of all individual nodes (rownames)
        
    
    """
    
    experiment = Output_path.split("/")[-2]
    n_protein_nodes = Molecular.shape[0]
    n_activity_nodes = n_protein_nodes + Phenotype.shape[0]
    n_x = n_activity_nodes + Perturbations.shape[0]

    config_json = construct_cellbox_json(experiment, n_protein_nodes, n_activity_nodes, n_x)

    activity_nodes = pd.merge(Molecular.T, Phenotype.T, left_index=True, right_index=True)
    expr_matrix = pd.merge(activity_nodes, Perturbations.T, left_index=True, right_index=True)

    
    #expr_matrix = pd.concat([Molecular, Phenotype, Perturbations]).T
    # Construct a dataframe where all initial values are zero in the shape of (# perturbations + # phenotypes, # samples)
    # Add corrresponding perturbation data
    pert_matrix = np.concatenate([np.zeros((Molecular.shape[0]+Phenotype.shape[0], Molecular.shape[1])),
                                  Perturbations]).T
    
    node_index = expr_matrix.columns.to_series()
    sample_order = expr_matrix.index.to_series()
    
    os.makedirs(Output_path, exist_ok=True)

    with open(os.path.join(Output_path, 'config.json'), 'w') as json_file:
        json.dump(config_json, json_file)

    np.savetxt(os.path.join(Output_path, 'expr_matr.csv'), expr_matrix.to_numpy(), delimiter=",")
    np.savetxt(os.path.join(Output_path, 'pert_matr.csv'), pert_matrix, delimiter=",")
    node_index.to_csv(os.path.join(Output_path, 'node_index.csv'), header=False, index=False)
    sample_order.to_csv(os.path.join(Output_path, 'sample_order.csv'), header=False, index=False)

def construct_cellbox_json(experiment, n_protein_nodes, n_activity_nodes, n_x):
    """
    Constructs a JSON object for the Cellbox experiment with the given parameters.

    Parameters:
    path (str): The base path for the files.
    n_protein_nodes (int): The number of protein nodes.
    n_activity_nodes (int): The number of activity nodes.
    n_x (int): The number of x nodes.

    Returns:
    dict: The constructed JSON object.
    """
    return {
        "experiment_id": experiment,
        "experiment_type": "random partition",
        "model": "CellBox",
        "sparse_data": False,
        "pert_file": f"Execution_files/{experiment}/pert_matr.csv",
        "expr_file": f"Execution_files/{experiment}/expr_matr.csv",
        "node_index_file": f"Execution_files/{experiment}/node_index.csv",
        "n_protein_nodes": n_protein_nodes,
        "n_activity_nodes": n_activity_nodes,
        "n_x": n_x,
        "trainset_ratio": 0.7,
        "validset_ratio": 0.8,
        "batchsize": 32,
        "add_noise_level": 0,
        "envelop_form": "tanh",
        "dT": 0.1,
        "envelop": 0,
        "ode_degree": 1,
        "ode_solver": "heun",
        "ode_last_steps": 2,
        "l1lambda": 1e-4,
        "l2lambda": 1e-4,
        "n_epoch": 10000,
        "n_iter": 10000,
        "n_iter_buffer": 50,
        "n_iter_patience": 100,
        "stages": [
            {
                "nT": 100,
                "sub_stages": [
                    {"lr_val": 0.1, "l1lambda": 0.01, "n_iter_patience": 1000},
                    {"lr_val": 0.01, "l1lambda": 0.01},
                    {"lr_val": 0.01, "l1lambda": 0.0001},
                    {"lr_val": 0.001, "l1lambda": 0.00001}
                ]
            },
            {
                "nT": 200,
                "sub_stages": [
                    {"lr_val": 0.001, "l1lambda": 0.0001}
                ]
            },
            {
                "nT": 400,
                "sub_stages": [
                    {"lr_val": 0.001, "l1lambda": 0.0001}
                ]
            }
        ],
        "export_verbose": 3,
        "ckpt_name": experiment
    }



In [90]:
A = pd.read_csv("../Data/ica/A.csv", index_col=0)
log2FC  = pd.read_csv("../Data/deseq2/Log2FC_All_Expression.csv", index_col=0)
logTPM_norm = pd.read_csv("../Data/ica/logTPM_counts_normalized_passingQC.csv", index_col=0)
tf_order=pd.read_csv('../Data/deseq2/tf_order.csv',header=None)
# From Puszynska & O'Shea 2017 (DOI: https://doi.org/10.7554/eLife.23210)
light_pert = pd.read_csv("../Data/literature_metadata/circadian_time_light_Puszynska_2017.csv")
light_pert = sin_cos_24hr_encoder(light_pert, 'Circadian_time_h')

## Load the genome annotation
ann = pd.read_csv("Selongatus_PCC7942_ref_tfs.csv")
ann.set_index('locus_tag', inplace=True)

# Load all predicted TFs
pred_TFs = pd.read_csv("../Data/annotation/All_predicted_TFs.csv")

# Load all known regulatory interactions
regs = pd.read_csv("TRN_complete.csv", index_col=0)

metadata = pd.read_csv('Metadata_Perturbation_Passing_QC.csv', index_col=0)
metadata.rename(columns={"Project_tag":"project", "Condition":"condition"}, inplace=True)
metadata['Ref_condition'] = metadata['project'] + "_" + metadata['Ref_condition']
metadata.head()

,BioProject,Biosample,Experiment,Bases,Bytes,create_date,Strain,Genotype,Experiment.1,condition,...,pH,Notes,Time_delta,Condition_pi,Project_condition_pi,Passed_Min_Reads,Passed_Global_Corr,passed_replicate_corr,passed_timeseries_corr,Overall_pass
SRA,,,,,,,,,,,,,,,,,,,,,
SRR807396,PRJNA196229,SAMN01999000,SRX259777,4.05 G,2.64 Gb,2015-11-15,Synechococcus elongatus PCC 7942 = FACHB-805,WT,RNA-seq,WT,...,NaN,NaN,4 days 04:00:00,WT_100h,FFA_WT_100h,True,True,True,False,True
SRR807397,PRJNA196229,SAMN01999001,SRX259778,2.71 G,1.77 Gb,2015-11-15,Synechococcus elongatus PCC 7942 = FACHB-805,WT,RNA-seq,WT,...,NaN,NaN,4 days 04:00:00,WT_100h,FFA_WT_100h,True,True,True,False,True
SRR807398,PRJNA196229,SAMN01999002,SRX259779,2.90 G,1.91 Gb,2015-11-15,Synechococcus elongatus PCC 7942 = FACHB-805,WT,RNA-seq,WT,...,NaN,NaN,4 days 04:00:00,WT_100h,FFA_WT_100h,True,True,True,False,True
SRR807399,PRJNA196229,SAMN01999003,SRX259780,3.19 G,2.08 Gb,2015-11-15,Synechococcus elongatus PCC 7942 = FACHB-805,WT,RNA-seq,WT,...,NaN,NaN,10 days 00:00:00,WT_240h,FFA_WT_240h,True,True,True,False,True
SRR807400,PRJNA196229,SAMN01999004,SRX259781,1.67 G,1.09 Gb,2015-11-15,Synechococcus elongatus PCC 7942 = FACHB-805,WT,RNA-seq,WT,...,NaN,NaN,10 days 00:00:00,WT_240h,FFA_WT_240h,True,True,True,False,True


In [87]:
markson_ann = pd.read_excel("../Data/literature_metadata/CircadianGenes_Markson_2013.xlsm", skiprows=1)
circadian_genes = markson_ann[markson_ann['Reproducibly circadian'] == 1]
print(f'Number of circadian genes: {circadian_genes.shape[0]}')
#circadian_genes = circadian_genes[~(circadian_genes['Chromosome or plasmid'] == 'pANS')]
circadian_genes.head(10)

## Manual mapping of circadian genes on PANL to alternative annotation using uniprot & Biocyc
## Single pANS gene not included
map_dict = {'HKK26_RS00110': 'Synpcc7942_B2626',
            'HKK26_RS00115': 'Synpcc7942_B2637',
            'HKK26_RS00120': 'Synpcc7942_B2648',
            'HKK26_RS00190': 'Synpcc7942_B2623',
            'HKK26_RS00235': 'Synpcc7942_B2655', 
            'HKK26_RS00240': 'Synpcc7942_B2632',
            'HKK26_RS00020': 'Synpcc7942_B2635'}

circadian_genes_up = ann[(ann.old_locus_tag.isin(circadian_genes['Gene ID'])) | (ann.index.isin(map_dict.keys()))]
print(f"Number of genes mapped to annotation: {circadian_genes_up.shape[0]}")
circadian_genes_up.head()

Number of circadian genes: 856
Number of genes mapped to annotation: 846


/Users/john002/miniconda3/envs/pimlomics_conda/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,start,end,length,strand,old_locus_tag,ncbi_protein,gene_name,gene_product,Eggnog_description,COG_category,...,ENTRAF_TF,ENTRAF_hmm_acc,ENTRAF_hmm_name,ENTRAF_Ortholog,DeepTFactor_TF,DeepTFactor_score,P2TF_TF,P2TF_class,P2TF_type,P2TF_description
locus_tag,,,,,,,,,,,,,,,,,,,,,
SYNPCC7942_RS00010,1262,2134,873,+,Synpcc7942_0002,WP_011243805.1,NaN,hypothetical protein,PFAM PRC-barrel domain,S,...,False,NaN,NaN,NaN,False,0.0282,False,NaN,NaN,NaN
SYNPCC7942_RS00020,4596,6077,1482,+,Synpcc7942_0004,WP_011243803.1,purF,amidophosphoribosyltransferase,Catalyzes the formation of phosphoribosylamine...,F,...,False,NaN,NaN,NaN,False,0.0137,False,NaN,NaN,NaN
SYNPCC7942_RS00025,6111,7706,1596,-,Synpcc7942_0005,WP_011377397.1,NaN,permease,secondary active sulfate transmembrane transpo...,P,...,False,NaN,NaN,NaN,False,0.0069,False,NaN,NaN,NaN
SYNPCC7942_RS00035,8189,8539,351,-,Synpcc7942_0007,WP_011377398.1,NaN,YtxH domain-containing protein,YtxH-like protein,S,...,False,NaN,NaN,NaN,False,0.0110,False,NaN,NaN,NaN
SYNPCC7942_RS00040,8536,8859,324,-,Synpcc7942_0008,WP_011377399.1,NaN,hypothetical protein,NaN,X,...,False,NaN,NaN,NaN,False,0.0096,False,NaN,NaN,NaN


In [96]:
#Load difference in gene expression data for TFs (1. Input into ICA)
light_pert_samples = light_pert[['SRA', 'Light', 'Hour_Sin', 'Hour_Cos']].set_index('SRA').T
light_pert_samples

oshea_samples_logTPM_norm_TFs = logTPM_norm.loc[pred_TFs.TFs, light_pert_samples.columns]
oshea_samples_logTPM_norm_TFs.index = oshea_samples_logTPM_norm_TFs.index.map('TF_{}'.format)
oshea_samples_logTPM_norm_TFs

light_pert_conditions = light_pert[['Condition_pi', 'Light', 'Hour_Sin', 'Hour_Cos']
                                  ].drop_duplicates().set_index('Condition_pi').T.drop(columns='wt_clearday_dawn_0.5h')

#Load log2FC data (2. Output from DESeq2)
oshea_samples_log2FC_TFs = log2FC.loc[pred_TFs.TFs, light_pert_conditions.columns]
oshea_samples_log2FC_TFs.index = oshea_samples_log2FC_TFs.index.map('TF_{}'.format)

# Modulon response by sample
oshea_IC_sample = A.loc[:, light_pert_samples.columns]

## Average the modulon resposne across conditions 
## wt_clearday_dawn_0.5h removed as it was used as the base condition for all differential expression comparisons

avg_A = []
for sample, group in light_pert.groupby('Condition_pi'):
    avg = oshea_IC_sample[group.SRA].mean(axis=1)
    avg_A.append(avg)
    
avg_A_df = pd.concat(avg_A, axis=1)
avg_A_df.columns = light_pert.groupby('Condition_pi').groups.keys()
avg_A_df = avg_A_df.drop('wt_clearday_dawn_0.5h', axis=1)
avg_A_df = avg_A_df.reindex(light_pert_conditions.columns.values, axis=1)
avg_A_df.head()

,wt_lowlight_dawn_0.5h,wt_lowlight_dawn_2h,wt_lowlight_dawn_4h,wt_lowlight_dawn_6h,wt_lowlight_dawn_8h,wt_lowlight_dawn_9h,wt_lowlight_dawn_10h,wt_lowlight_dawn_12h,wt_clearday_dawn_2h,wt_clearday_dawn_4h,...,wt_highlight_pulse_9.25h,wt_highlight_pulse_9.5h,wt_highlight_pulse_10h,wt_shade_pulse_8h,wt_shade_pulse_8.25h,wt_shade_pulse_8.5h,wt_shade_pulse_9h,wt_shade_pulse_9.25h,wt_shade_pulse_9.5h,wt_shade_pulse_10h
0,6.115376,0.713443,5.495462,5.273049,4.442937,3.272571,2.836849,3.316532,-0.531633,3.782513,...,-0.225281,0.573574,2.066296,2.203895,0.303699,0.656124,1.037911,4.006790,2.951316,1.298457
1,0.248676,-0.937995,-5.065755,-10.140796,-12.870681,-11.926588,-11.279455,-10.108009,0.518376,-2.770383,...,-20.002566,-19.826402,-15.607871,-12.388667,-19.352622,-20.212467,-18.382456,-13.501904,-13.467382,-14.270156
2,3.133630,0.480528,-2.304670,-5.256915,-5.876160,-4.045070,-3.457322,-2.933155,6.389302,3.143030,...,-3.286208,-6.702305,-6.648800,-1.525668,-6.582757,-9.393851,-9.860173,-6.932270,-2.141695,-1.488718
3,1.016743,-0.677561,1.119265,3.072089,4.615046,5.071194,4.112884,4.287350,0.212489,2.811278,...,4.306938,0.986831,4.312314,1.790151,7.804377,5.035021,5.449592,0.279242,1.924907,3.072254
4,1.744702,3.704905,2.669867,2.326103,2.262399,3.160924,1.531659,3.776586,2.761959,3.731732,...,2.703554,2.016426,2.577973,3.315887,2.580680,3.114974,2.111801,0.803972,0.145796,1.672850


# Module model

In [ ]:
circadian_tfs = set(circadian_genes_up.index).intersection(set(pred_TFs.TFs))
add_regs = ['RpaA', 'RpaB', 'rre1', 'HrcA', 'NtcA']
add_regs_locus_tags = regs[regs.Regulator_Name.isin(add_regs)].regulatoryGene.unique()


print(f'Known Regulators identified in circadian phase: {regs[regs.regulatoryGene.isin(circadian_tfs)].Regulator_Name.unique()}')
print(f'Inclusion of known regulators: {regs[regs.regulatoryGene.isin(circadian_tfs.union(set(add_regs_locus_tags)))].Regulator_Name.unique()}')

circ_log2FC_TFs = oshea_samples_log2FC_TFs.loc[list(["TF_" + TF for TF in circadian_tfs.union(set(add_regs_locus_tags))]),:]
circ_log2FC_TFs = circ_log2FC_TFs.reindex(tf_order[0].values)
construct_cellbox_input(light_pert_conditions, circ_log2FC_TFs, avg_A_df, '../Execution_files/Cellbox_Module')


Known Regulators identified in circadian phase: ['RpoD6' 'RpoD5' 'SigF2' 'CcmR' 'kaiC' 'pex']
Inclusion of known regulators: ['RpaA' 'RpaB' 'RpoD6' 'RpoD5' 'SigF2' 'NtcA' 'CcmR' 'HrcA' 'kaiC' 'rre1'
 'pex']


# Gene model

In [ ]:
circ_log2FC_genes = log2FC[log2FC.index.isin(circadian_genes_up.index)]

construct_cellbox_input(light_pert_conditions, circ_log2FC_TFs, circ_log2FC_genes, 
                        "../Execution_files/Cellbox_Gene")
